<a href="https://colab.research.google.com/github/werd0n4/hand-gesture-classification/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

---


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import os
import cv2
import glob
import random
from matplotlib import pyplot as plt

# Constant parameters

---



In [4]:
horiz_x = 174
vert_y = 131
imgs_train_path = '/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Train/'
imgs_test_path = '/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Test/'

In [5]:

class_names = {
    0: "Undefined",
    1: "1",
    2: "2",
    3: "3",
    4: "4",
    5: "5",
    6: "A",
    7: "B",
    8: "C",
    9: "D",
    10: "E",
    11: "F",
    12: "G",
    13: "H",
    14: "I",
    15: "K",
    16: "L",
    17: "M",
    18: "N",
    19: "O",
    20: "P",
    21: "R",
    22: "S",
    23: "T",
    24: "U",
    25: "W",
    26: "Y",
    27: "Z"
}


# Auxiliary functions

---


In [6]:

def resize(path):
    img_counter = 0

    for dirname in os.listdir(path): 
        for filename in os.listdir(os.path.join(path, dirname)):
            image_path = os.path.join(path, dirname, filename)
            # print(image_path)
            img = cv2.imread(image_path)
            resized_img = cv2.resize(img, (horiz_x, vert_y))
            cv2.imwrite(image_path, resized_img)
            img_counter += 1
    
    print('Images in set: ' + str(img_counter))


In [7]:

def sanity_check(path):
    counter = 0

    for dirname in os.listdir(path): 
        for filename in os.listdir(os.path.join(path, dirname)):
            image_path = os.path.join(path, dirname, filename)
            img = cv2.imread(image_path)
            if img.shape != (horiz_x, vert_y, 3):
                counter += 1

    print('Sanity result: ' + str(counter))

In [8]:

def show_img(index, X, Y):
    # plt.imshow(X[index])
    plt.imshow(cv2.cvtColor(X[index],cv2.COLOR_BGR2RGB).astype('float32'))
    plt.show()
    nmb = int(np.where(Y[index] == 1)[0])
    print("On image: " + class_names[nmb])

## Load dataset

In [16]:

def load_dataset():
    trainlist = glob.glob(f'{imgs_train_path}/*/*')
    testlist = glob.glob(f'{imgs_test_path}/*/*')
    X_train = np.array( [np.array(cv2.normalize(cv2.imread(fname), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)) for fname in trainlist] )
    X_test = np.array( [np.array(cv2.normalize(cv2.imread(fname), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)) for fname in testlist] )

    # Y_train = np.array([np.zeros(28) for fname in trainlist])
    Y_train = np.array([0 for fname in trainlist])
    for i,fname in enumerate(trainlist):
        img_id = fname.split('/')[7]
        img_id = img_id.split('_')[0]
        # Y_train[i][img_id] = 1
        Y_train[i] = img_id


    # Y_test = np.array([np.zeros(28) for fname in testlist])
    Y_test = np.array([0 for fname in testlist])
    for i,fname in enumerate(testlist):
        img_id = fname.split('/')[7]
        img_id = img_id.split('_')[0]
        # Y_test[i][img_id] = 1
        Y_test[i] = img_id

    return X_train, Y_train, X_test, Y_test


## Create network model

In [ ]:

def create_model():
    model = Sequential()
    
    # CONVOLUTIONAL LAYER
    model.add(Conv2D(
        filters=6, 
        kernel_size=(5,5), 
        input_shape=(131, 174, 3), 
        activation='relu'
    ))

    # POOLING LAYER
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER
    model.add(Conv2D(
        filters=16, 
        kernel_size=(5,5), 
        input_shape=(62, 84, 6), 
        activation='relu'
    ))

    # POOLING LAYER
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(27, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam', 
        metrics=['accuracy']
    )

    return model


# Loading dataset

---


In [ ]:
# Uncomment before first run on dataset 
# resize(imgs_dir_path)
# sanity_check(imgs_dir_path)


In [18]:
X_train, Y_train, X_test, Y_train = load_dataset()

In [ ]:
classes_count = count_classes()

In [ ]:
print(X.shape)
print(Y.shape)
print(classes_count)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
# One hot encoding
Y_cat_train = to_categorical(Y_train)
Y_cat_test = to_categorical(Y_test)

# Create and train model

---


In [ ]:
CNN = create_model()
CNN.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=1)

In [ ]:
CNN.fit(X_train, Y_cat_train, epochs=32, validation_data=(X_test, Y_cat_test), callbacks=[early_stop])

# Results

---


In [ ]:
metrics = pd.DataFrame(CNN.history.history)
metrics

## Single image prediction

In [ ]:
my_hand = X_test[0]
show_img(0, X_test, Y_cat_test)

In [ ]:
my_hand = my_hand.reshape(1, 131, 174, 3)
result = CNN.predict_classes(my_hand)
result = int(result)
print('CNN says: ' + class_names[result])

## Predictions for individual classes

In [ ]:
predictions = CNN.predict_classes(X_test)

In [ ]:
print(classification_report(Y_test,predictions))